## <span style="font-family:cursive;text-align:center">⬇️ Loading data</span>

In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
df.dropna(inplace=True)
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


## <span style="font-family:cursive;text-align:center">⬇️ Installing ChromaDB and sentence_transformers</span> 

In [2]:
!pip install chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 10.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.2-cp310-cp310-linux_x86_64.whl size=194

<span style="font-family:cursive;text-align:center">'all-MiniLM-L6-v2' is a pre-trained model that can transform sentences or text snippets into numerical embeddings. These embeddings can be useful for a variety of NLP tasks, leveraging the model's understanding of semantics and context captured during its training on diverse textual data.</span>

In [3]:
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('all-MiniLM-L6-v2')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

<span style="font-family:cursive;text-align:center">Writing a loop to store our description ids and metadatas in lists so that can be added to our vector db directly</span>

In [13]:
documents = []
metadatas = []
ids = []

for i in range(len(df)):
    documents.append(df.iloc[i,4])
    metadatas.append({'link': df.iloc[i,3],'Date':df.iloc[i,1]})
    ids.append(str(i + 1))

<span style="font-family:cursive;text-align:center">ChromaDB client can be accessed with `chromadb.Client()` which can create and terminate a new database.</span>

In [15]:
import chromadb
client = chromadb.Client()

<span style="font-family:cursive;text-align:center">Here lets start by creating a collection called as Kaggle and add the required inputs to it</span>

In [16]:
vector_collection = client.create_collection("kaggle")

vector_collection.add(
    documents=description,
    metadatas=metadatas,
    ids=ids
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 29.0MiB/s]


<span style="font-family:cursive;text-align:center">Now one can query on the vector database directly using the `.query()` function</span>

In [45]:
results = vector_collection.query(
    query_texts=["Ukraine Minister"],
    n_results=5
)

for i in range(len(results)):
    print('Id:',results['ids'][0][i])
    print('Metadata:',results['metadatas'][0][i])
    print('Description:',results['documents'][0][i])
    print('\n')

Id: 10577
Metadata: {'Date': 'Sat, 19 Nov 2022 19:53:43 GMT', 'link': 'https://www.bbc.co.uk/news/world-europe-63687045?at_medium=RSS&at_campaign=KARANGA'}
Description: The UK prime minister has travelled to Ukraine for the first time since taking office.


Id: 18302
Metadata: {'Date': 'Wed, 14 Jun 2023 10:10:45 GMT', 'link': 'https://www.bbc.co.uk/news/world-europe-65901723?at_medium=RSS&at_campaign=KARANGA'}
Description: Gen Valery Zaluzhny is Ukraine's handpicked army boss and mastermind of the unlikely successes over Russia.


Id: 4306
Metadata: {'Date': 'Thu, 16 Jun 2022 17:15:58 GMT', 'link': 'https://www.bbc.co.uk/news/61829953?at_medium=RSS&at_campaign=KARANGA'}
Description: Russia's foreign minister has been speaking to the BBC about the war in Ukraine.


Id: 1373
Metadata: {'Date': 'Tue, 05 Apr 2022 23:01:16 GMT', 'link': 'https://www.bbc.co.uk/news/world-europe-60996098?at_medium=RSS&at_campaign=KARANGA'}
Description: A Ukrainian man has to choose between his family and his 

<span style="font-family:cursive;text-align:center">For filtering certain things lets say I want document that contains Russia but should contain Olena Zelenska in the description then we can use `where_document` to search for that particularly</span>

In [56]:
result = vector_collection.query(
    query_texts=["Russia"],
    n_results=5,
    where_document={"$contains":"Olena Zelenska"}
)

for i in range(len(result)):
    print('Id:',result['ids'][0][i])
    print('Metadata:',result['metadatas'][0][i])
    print('Description:',result['documents'][0][i])
    print('\n')

Id: 11013
Metadata: {'Date': 'Tue, 29 Nov 2022 18:27:41 GMT', 'link': 'https://www.bbc.co.uk/news/uk-63797240?at_medium=RSS&at_campaign=KARANGA'}
Description: Speaking in the Commons, Olena Zelenska compared Ukraine's plight to that of the UK during World War Two.


Id: 10851
Metadata: {'Date': 'Thu, 24 Nov 2022 23:11:08 GMT', 'link': 'https://www.bbc.co.uk/news/world-europe-63743657?at_medium=RSS&at_campaign=KARANGA'}
Description: Olena Zelenska tells the BBC's Lyse Doucet cold and blackouts are not the worst things Ukraine has faced.


Id: 16611
Metadata: {'Date': 'Thu, 04 May 2023 15:44:12 GMT', 'link': 'https://www.bbc.co.uk/news/uk-65487352?at_medium=RSS&at_campaign=KARANGA'}
Description: Olena Zelenska gets a warm welcome to the UK from Akshata Murty on the steps of Downing Street.


Id: 3249
Metadata: {'Date': 'Sat, 21 May 2022 19:53:21 GMT', 'link': 'https://www.bbc.co.uk/news/world-europe-61535346?at_medium=RSS&at_campaign=KARANGA'}
Description: Olena Zelenska opens up about t

<span style="font-family:cursive;text-align:center">To count the number of items in a collection use `.count`</span>

In [57]:
vector_collection.count()

20536

<span style="font-family:cursive;text-align:center">To get first 5 items in a collection use `.peek`</span>

In [58]:
vector_collection.peek()

{'ids': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'embeddings': [[0.03998638689517975,
   0.05682724341750145,
   -0.014636773616075516,
   -0.013452598825097084,
   0.013239418156445026,
   -0.033963076770305634,
   0.057396139949560165,
   -0.10155896842479706,
   0.030208218842744827,
   0.00844224076718092,
   0.01964818313717842,
   0.07120300084352493,
   0.0864400640130043,
   -0.01188740599900484,
   -0.05490976572036743,
   -0.0034800898283720016,
   -0.09441806375980377,
   0.005166208371520042,
   -0.016515377908945084,
   -0.0005576781113632023,
   0.05790025368332863,
   -0.012165237218141556,
   0.12236945331096649,
   0.06683322042226791,
   0.006029635202139616,
   -0.026039553806185722,
   0.057700831443071365,
   0.055718980729579926,
   -0.05125129967927933,
   0.020276207476854324,
   0.01625264808535576,
   -0.043419744819402695,
   -0.006570351310074329,
   0.049749087542295456,
   -0.030902499333024025,
   0.02094368264079094,
   -0.002221364295110106

<span style="font-family:cursive;text-align:center">To delete a collection use `.delete`</span>

In [ ]:
# vector_collection.delete()